In [8]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

def drop_vars(df):
    tmp=df.shape[1]
    df = df[df.columns[[True]+list((df.var()!=0))]]
    print('0 var:',tmp-df.shape[1])
    
    return df
train = drop_vars(train)

0 var: 256


In [5]:
pca = PCA(n_components=2000, svd_solver='arpack')
train_X = pca.fit_transform(train[train.columns[2:]])
test_X = pca.transform(test[train.columns[2:]])
train_y=train['target']
    

## Test Improvement

In [25]:
X=train_X
y=np.log1p(train_y)

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=203)

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

def root_mean_squared_error(y_true, y_pred):
        return np.sqrt(np.mean((y_pred - y_true)**2)) 

clf = ElasticNet(random_state=23,alpha=5)
clf.fit(x_train, y_train)

y_true,y_pred = y_test,clf.predict(x_test) 
print(root_mean_squared_error(y_pred, y_true))


1.7418805964740947


In [26]:
scal=StandardScaler()
x_train = np.c_[x_train,scal.fit_transform(clf.predict(x_train).flatten().reshape(-1,1)).flatten()]
x_test = np.c_[x_test,scal.transform(clf.predict(x_test).flatten().reshape(-1,1)).flatten()]

model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.1,decay=0.0001))


checkp = ModelCheckpoint(filepath='weights.hdf5')
lrred = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='val_loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=1000,
        batch_size=400,
        validation_data=(x_test, y_test),
        callbacks=cbs)

Train on 3567 samples, validate on 892 samples
Epoch 1/1000
3567/3567 [==============================] - 0s 86us/step - loss: 10.8089 - val_loss: 7.7118
Epoch 2/1000
3567/3567 [==============================] - 0s 31us/step - loss: 8.0340 - val_loss: 5.0894
Epoch 3/1000
3567/3567 [==============================] - 0s 32us/step - loss: 7.0360 - val_loss: 3.9692
Epoch 4/1000
3567/3567 [==============================] - 0s 31us/step - loss: 6.8750 - val_loss: 3.6384
Epoch 5/1000
3567/3567 [==============================] - 0s 31us/step - loss: 6.3612 - val_loss: 4.8525
Epoch 6/1000
3567/3567 [==============================] - 0s 30us/step - loss: 5.4710 - val_loss: 3.3750
Epoch 7/1000
3567/3567 [==============================] - 0s 31us/step - loss: 5.3872 - val_loss: 4.4031
Epoch 8/1000
3567/3567 [==============================] - 0s 31us/step - loss: 4.7896 - val_loss: 5.2269
Epoch 9/1000
3567/3567 [==============================] - 0s 30us/step - loss: 4.3664 - val_loss: 2.6272
Epoch 1

3567/3567 [==============================] - 0s 30us/step - loss: 1.2933 - val_loss: 1.4070
Epoch 77/1000
3567/3567 [==============================] - 0s 30us/step - loss: 1.2914 - val_loss: 1.4088
Epoch 78/1000
3567/3567 [==============================] - 0s 31us/step - loss: 1.2836 - val_loss: 1.4097
Epoch 79/1000
3567/3567 [==============================] - 0s 30us/step - loss: 1.2914 - val_loss: 1.4079
Epoch 80/1000
3567/3567 [==============================] - 0s 30us/step - loss: 1.2947 - val_loss: 1.4091
Epoch 81/1000
3567/3567 [==============================] - 0s 32us/step - loss: 1.2939 - val_loss: 1.4066
Epoch 82/1000
3567/3567 [==============================] - 0s 31us/step - loss: 1.3092 - val_loss: 1.4091
Epoch 83/1000
3567/3567 [==============================] - 0s 31us/step - loss: 1.2900 - val_loss: 1.4094
Epoch 84/1000
3567/3567 [==============================] - 0s 31us/step - loss: 1.2894 - val_loss: 1.4073
Epoch 85/1000
3567/3567 [==============================] - 0

## NN+ElNet - Actual

In [36]:
x_train=train_X
y_train=np.log1p(train_y)
x_test=test_X

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

clf = ElasticNet(random_state=23,alpha=4)
clf.fit(x_train, y_train)

scal=StandardScaler()
x_train = np.c_[x_train,scal.fit_transform(clf.predict(x_train).flatten().reshape(-1,1)).flatten()]
x_test = np.c_[x_test,scal.transform(clf.predict(x_test).flatten().reshape(-1,1)).flatten()]

model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.1,decay=0.0001))


checkp = ModelCheckpoint(filepath='weights.hdf5')
lrred = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=1000,
        batch_size=400,
        callbacks=cbs)

Epoch 1/1000
4459/4459 [==============================] - 0s 70us/step - loss: 15.3520
Epoch 2/1000
4459/4459 [==============================] - 0s 27us/step - loss: 11.0362
Epoch 3/1000
4459/4459 [==============================] - 0s 27us/step - loss: 9.1567
Epoch 4/1000
4459/4459 [==============================] - 0s 27us/step - loss: 8.0253
Epoch 5/1000
4459/4459 [==============================] - 0s 27us/step - loss: 7.1107
Epoch 6/1000
4459/4459 [==============================] - 0s 26us/step - loss: 6.0591
Epoch 7/1000
4459/4459 [==============================] - 0s 27us/step - loss: 5.3447
Epoch 8/1000
4459/4459 [==============================] - 0s 27us/step - loss: 4.6557
Epoch 9/1000
4459/4459 [==============================] - 0s 27us/step - loss: 4.0785
Epoch 10/1000
4459/4459 [==============================] - 0s 27us/step - loss: 3.3039
Epoch 11/1000
4459/4459 [==============================] - 0s 27us/step - loss: 2.7383
Epoch 12/1000
4459/4459 [=========================

4459/4459 [==============================] - 0s 28us/step - loss: 1.2825
Epoch 96/1000
4459/4459 [==============================] - 0s 28us/step - loss: 1.2842
Epoch 97/1000
4459/4459 [==============================] - 0s 27us/step - loss: 1.2719
Epoch 98/1000
4459/4459 [==============================] - 0s 28us/step - loss: 1.2740
Epoch 99/1000
4459/4459 [==============================] - 0s 28us/step - loss: 1.2890
Epoch 100/1000
4459/4459 [==============================] - 0s 37us/step - loss: 1.2703
Epoch 101/1000
4459/4459 [==============================] - 0s 27us/step - loss: 1.2703
Epoch 102/1000
4459/4459 [==============================] - 0s 25us/step - loss: 1.2821
Epoch 103/1000
4459/4459 [==============================] - 0s 26us/step - loss: 1.2767
Epoch 104/1000
4459/4459 [==============================] - 0s 26us/step - loss: 1.2856
Epoch 105/1000
4459/4459 [==============================] - 0s 26us/step - loss: 1.2729
Epoch 106/1000
4459/4459 [=========================

4459/4459 [==============================] - 0s 26us/step - loss: 1.2617
Epoch 189/1000
4459/4459 [==============================] - 0s 26us/step - loss: 1.2636
Epoch 190/1000
4459/4459 [==============================] - 0s 27us/step - loss: 1.2595
Epoch 191/1000
4459/4459 [==============================] - 0s 27us/step - loss: 1.2493
Epoch 192/1000
4459/4459 [==============================] - 0s 27us/step - loss: 1.2638
Epoch 193/1000
4459/4459 [==============================] - 0s 28us/step - loss: 1.2439
Epoch 194/1000
4459/4459 [==============================] - 0s 27us/step - loss: 1.2717
Epoch 195/1000
4459/4459 [==============================] - 0s 26us/step - loss: 1.2567
Epoch 196/1000
4459/4459 [==============================] - 0s 27us/step - loss: 1.2630
Epoch 197/1000
4459/4459 [==============================] - 0s 27us/step - loss: 1.2487
Epoch 198/1000
4459/4459 [==============================] - 0s 27us/step - loss: 1.2623
Epoch 199/1000
4459/4459 [=====================

In [38]:
test['ID'].shape

(49342,)

In [39]:
predictions=pd.DataFrame({'ID':test['ID'],'target':np.expm1(model.predict(x_test, verbose=1).flatten())})
print(predictions.head())
predictions.to_csv('pred_stack.csv',index=False)

49342/49342 [==============================] - 1s 26us/step
          ID        target
0  000137c73  2.114571e+08
1  00021489f  1.737022e+06
2  0004d7953  3.438895e+06
3  00056a333  3.723570e+06
4  00056d8eb  2.094869e+06
